# Surprise 패키지를 이용한 추천시스템 구현

In [1]:
import surprise
print(surprise.__version__)

from surprise import SVD, Reader, Dataset, accuracy 
from surprise.model_selection import train_test_split, cross_validate, GridSearchCV
from surprise.dataset import DatasetAutoFolds
import pandas as pd
import numpy as np

1.1.1


### 추천 알고리즘 평가

#### 데이터 준비

In [3]:
# surprise의 Reader 클래스는 헤더가 없는 CSV 형식의 평점 화일을 요구함
pd.read_csv('ratings.csv').to_csv('ratings_noh.csv', index=False, header=False)

# 평점화일이 user, item, rating, timestamp 순으로 필드가 구성되어 있는다는 것을 알려줘야 함
# 또한 최소 평점과 최대 평점을 지정해야 함
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data = Dataset.load_from_file('ratings_noh.csv', reader)

In [4]:
#ratings = pd.read_csv('ratings.csv')
#reader = Reader(rating_scale=(0.5, 5.0))
#data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

#### 학습과 평가용으로 데이터 분리

In [5]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

#### 추천 알고리즘 학습

In [6]:
model = SVD(n_factors=50, n_epochs=20, biased=True, random_state=0)
model.fit(trainset)

#### 추천 성능 평가

In [7]:
predictions = model.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

#### 교차 검증

In [8]:
model = SVD(random_state=0) 
cross_validate(model, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.8879  0.8778  0.8800  0.8819  0.0043  
MAE (testset)     0.6829  0.6743  0.6773  0.6782  0.0036  
Fit time          3.30    3.40    3.62    3.44    0.13    
Test time         0.21    0.21    0.25    0.23    0.02    


{'test_rmse': array([0.88787516, 0.87779416, 0.88004853]),
 'test_mae': array([0.68287814, 0.67430199, 0.67729127]),
 'fit_time': (3.301173210144043, 3.4038915634155273, 3.619288682937622),
 'test_time': (0.21144723892211914, 0.21243715286254883, 0.25333094596862793)}

#### 하이퍼 파라미터 튜닝

In [9]:
# 최적화할 파라미터들을 딕셔너리 형태로 지정. 
param_grid = {'n_epochs': [20, 40], 'n_factors': [50, 100] }

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse 로 수행 하도록 GridSearchCV 구성
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
grid_search.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(grid_search.best_score['rmse'])
print(grid_search.best_params['rmse'])

0.8767994747960896
{'n_epochs': 20, 'n_factors': 50}


### 추천 아이템 생성 

#### 데이터 준비

In [10]:
# Ratings Matrix 생성
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
rating_movies = pd.merge(ratings, movies, on='movieId')
ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title', fill_value=0)

In [11]:
# 학습 데이터 생성
# train_test_split( )으로 분리되지 않는 Dataset에 fit( )을 호출하면 오류 발생
# 전체 데이터를 학습 데이터로 사용하려면 DatasetAutoFolds 클래스를 사용해야 함 
data_folds = DatasetAutoFolds(ratings_file='ratings_noh.csv', reader=reader)

#전체 데이터를 학습데이터로 생성함. 
trainset = data_folds.build_full_trainset()

#### 영화 평점 예측

In [12]:
model = SVD(n_epochs=20, n_factors=50, random_state=0)
model.fit(trainset)

#### Top-N 영화 추천

In [13]:
# 추천 영화의 수 설정
N = 10
uid = 9

In [14]:
#testset = trainset.build_anti_testset()
#predictions = model.test(testset)

ratings_pred = np.zeros((1, ratings_matrix.shape[1]))
for i in range(ratings_matrix.shape[1]):
    ratings_pred[0,i] = model.predict(str(uid), str(i))[3]
    
ratings_pred = pd.DataFrame(ratings_pred, columns=ratings_matrix.columns)
ratings_pred 

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
0,3.195364,3.639802,3.070854,2.991252,2.758491,2.65381,3.762764,2.883779,2.986577,2.788971,...,3.195364,3.195364,3.195364,3.195364,3.195364,3.195364,3.195364,3.195364,3.195364,3.195364


In [15]:
# id로 지정된 사용자의 모든 영화정보 추출하여 Series로 반환함
# 반환된 user_rating은 영화명(title)을 index로 가지는 Series 객체임 
user_rating = ratings_matrix.loc[uid,:]
    
# user_rating이 0보다 크면 기존에 관람한 영화임. 대상 index를 추출하여 list로 만듬
already_seen = user_rating[user_rating > 0].index.tolist()
   
# list comprehension으로 already_seen에 해당하는 movie는 movies_list에서 제외함
unseen_list = [movie for movie in ratings_matrix.columns.tolist() if movie not in already_seen]

# unseen_list에서 가장 평점이 높은 N개의 영화를 추천함 
recomm_items = ratings_pred.loc[0, unseen_list].sort_values(ascending=False)[:N]
list(recomm_items.index)

['Battle Royale (Batoru rowaiaru) (2000)',
 'African Cats (2011)',
 'Akira (1988)',
 'Bloodsport III (1996)',
 'Bloodsucking Bastards (2015)',
 '1492: Conquest of Paradise (1992)',
 'Biutiful (2010)',
 'Blue Exorcist: The Movie (2012)',
 'Legend (2015)',
 'Blue Juice (1995)']

# End